# 🥗 Safe-Plate Scout: The Forensic Dietary Concierge

**Capstone Submission for Google AI Agents Intensive 2025**
**Track:** Concierge Agents

## 💡 The Why: Solving "Dietary Anxiety"

For the 10% of the population with strict dietary needs (Celiacs, severe allergies, religious restrictions), finding food isn't just about taste—it's about safety. **Safe-Plate Scout** is a multi-agent system that acts as a forensic dietary concierge. Unlike standard search engines that prioritize popularity, this agent prioritizes **safety verification**.

It uses a "Trust but Verify" architecture: one agent finds the food, and a second "Auditor" agent aggressively checks it for safety risks based on the user's specific profile.

## 🛠️ Key Agentic Features Implemented

This project demonstrates the following advanced agent concepts required for the capstone:

### 1\. Multi-Agent System (Sequential & Hierarchical)

  * **Concept:** We use a team of specialized agents rather than a single generalist model. A **Router** delegates tasks to specialists, and an **Auditor** reviews the work.
  * **Implementation:**
      * `router_agent`: Classifies intent (Restaurant vs. Grocery).
      * `restaurant_vetter` / `grocery_vetter`: Specialists that perform the actual research.
      * `auditor_agent`: A distinct persona that reviews the findings.
  * **Where in Code:** Defined in `agent_engine.py` (Lines 77-133). The orchestration logic in `process_user_request` (Lines 136-193) manages the sequential hand-offs.

### 2\. Built-in Tools (Google Search Grounding)

  * **Concept:** The agents are not limited to their training data. They access real-time information from the web to find current menus, ingredients, and allergen statements.
  * **Implementation:** We utilize the native **Google Search Tool** provided by the Google GenAI SDK.
  * **Where in Code:** Configured in `agent_engine.py`:
    ```python
    search_tool = types.Tool(google_search=types.GoogleSearch())
    ```
    This tool is injected dynamically into the vetting agents during execution.

### 3\. Agent Evaluation (LLM-as-a-Judge)

  * **Concept:** Instead of blindly trusting the search results, we implement a self-correction/evaluation step *before* showing the user the answer.
  * **Implementation:** The `auditor_agent` receives the raw findings from the vetter. It is instructed to act as a "Quality Assurance Auditor," assigning a numerical **Safety Confidence Score (0-100)** and a "Green/Red Light" verdict.
  * **Where in Code:** The `auditor_agent` definition (Lines 122-133) and the final validation step in the orchestrator (Lines 180-190).

### 4\. Context Engineering

  * **Concept:** We dynamically construct prompts to enforce strict constraints (Geofencing) and persona alignment.
  * **Implementation:** The prompt template uses f-strings to inject the `Target City/Location` and strictly enforce a "City Limits" rule to prevent the LLM from hallucinating nearby towns as valid results.
  * **Where in Code:** The `localized_prompt` construction in `agent_engine.py` (Lines 153-169).

### 5\. Agent Deployment

  * **Concept:** Moving from a notebook prototype to a scalable web service.
  * **Implementation:** The agent is wrapped in a **FastAPI** backend and containerized using **Docker** for deployment on **Google Cloud Run**.
  * **Where in Code:** `main.py` (API Server) and `Dockerfile`.

## 🏗️ Architecture

1.  **Frontend (HTML/JS):** A responsive web interface (Tailwind CSS) that captures the user's Location, Dietary Profile, and Mission.
2.  **Backend (FastAPI):** A Python API that handles requests and serves the agent.
3.  **The "Brain" (Gemini 2.0 Flash):**
      * **Input:** "Find gluten-free pizza in Chicago."
      * **Step 1 (Router):** Identifies intent -\> "RESTAURANT".
      * **Step 2 (Vetter):** Searches Google for 10 candidates, filters for safety evidence, and selects the Top 6.
      * **Step 3 (Auditor):** Reviews the selected list and assigns a final Safety Score.
      * **Output:** A structured JSON payload containing the vetted list and the auditor's scorecard.

## 🚀 Setup & Usage

### Prerequisites

  * Python 3.10+
  * Google Cloud API Key (Gemini)

### Installation

1.  Clone the repository.
2.  Create a `.env` file and add your key: `GOOGLE_API_KEY="your_key_here"`
3.  Install dependencies:
    ```bash
    pip install -r requirements.txt
    ```

### Running Locally

```bash
uvicorn main:app --reload
```

Visit `http://127.0.0.1:8000` in your browser.

### Deployment (Google Cloud Run)

```bash
gcloud run deploy safe-plate-scout --source . --set-env-vars GOOGLE_API_KEY=YOUR_KEY
```

In [1]:
# [SETUP] Install the Google GenAI SDK and Agent Development Kit
!pip install -U google-genai google-adk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.48.0
    Uninstalling google-genai-1.48.0:
      Successfully uninstalled google-genai-1.48.0
  Attempting uninstall: google-adk
    Found existing installation: google-adk 1.18.0
    

In [2]:
import os
import json
from google import genai
from google.genai import types
from google.adk.agents import LlmAgent
from kaggle_secrets import UserSecretsClient

# [CONFIGURATION]
# Retrieve API Key from Kaggle Secrets
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    client = genai.Client(api_key=GOOGLE_API_KEY)
    print("✅ API Key loaded and Client initialized.")
except Exception as e:
    print("⚠️ Error loading API Key. Please ensure 'GOOGLE_API_KEY' is in Kaggle Secrets.")

# Model Strategy
FAST_MODEL = "gemini-2.0-flash" 
SMART_MODEL = "gemini-2.0-flash"

print(f"✅ Using Models: {FAST_MODEL}")

✅ API Key loaded and Client initialized.
✅ Using Models: gemini-2.0-flash


In [3]:
# --- SCHEMAS & CONFIG ---

# 1. Tool Definition (Google Search)
search_tool = types.Tool(
    google_search=types.GoogleSearch()
)

# 2. Structured Output Schemas
rec_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "name": {"type": "STRING"},
            "address": {"type": "STRING"},
            "website_url": {"type": "STRING"},
            "safe_items": {"type": "ARRAY", "items": {"type": "STRING"}},
            "safety_score": {"type": "INTEGER"},
            "reasoning": {"type": "STRING"}
        },
        "required": ["name", "safe_items", "reasoning"]
    }
}

auditor_schema = {
    "type": "OBJECT",
    "properties": {
        "overall_score": {"type": "INTEGER"},
        "headline": {"type": "STRING"},
        "summary_notes": {"type": "ARRAY", "items": {"type": "STRING"}}
    }
}

# --- HELPER FUNCTION ---
def query_agent_with_runner(agent, prompt, tools=None, json_mode=False, schema=None):
    """
    Executes the agent using the google-genai SDK.
    """
    config_args = {
        "tools": tools,
        "temperature": 0.2, 
        "safety_settings": [
            types.SafetySetting(
                category="HARM_CATEGORY_DANGEROUS_CONTENT",
                threshold="BLOCK_ONLY_HIGH"
            ),
        ]
    }

    if json_mode:
        config_args["response_mime_type"] = "application/json"
        if schema:
            config_args["response_schema"] = schema

    generate_config = types.GenerateContentConfig(**config_args)
    
    try:
        # Use the global client initialized in Cell 3
        response = client.models.generate_content(
            model=agent.model,
            contents=prompt,
            config=generate_config
        )
        if not response.text:
            return "[]" if json_mode and schema == rec_schema else "{}"
        return response.text
    except Exception as e:
        return f"⚠️ Agent Error: {str(e)}"

# --- AGENT DEFINITIONS ---

router_agent = LlmAgent(
    name="intent_router",
    model=FAST_MODEL,
    description="Classifies user intent.",
    instruction="Determine if query is RESTAURANT or GROCERY. Output one word."
)

restaurant_vetter = LlmAgent(
    name="restaurant_vetter",
    model=FAST_MODEL,
    description="Finds and checks restaurants.",
    instruction="""
    You are a dietary safety officer. 
    1. SEARCH GOAL: Find enough valid options to fill a list of 6.
    2. CHECK LOCATION: Reject any result that is NOT in the requested city.
    
    3. SCORING RUBRIC (EVIDENCE-BASED):
       - 95-100 (HIGH SAFETY): "Dedicated Gluten Free Menu", "Dedicated Kitchen".
       - 80-94 (MODERATE): "Gluten Friendly" options, good reviews.
       - <80 (LOW): Unclear safety protocols.

    4. EVIDENCE: Extract quotes for reasoning.
    5. Output JSON.
    """
)

grocery_vetter = LlmAgent(
    name="grocery_vetter",
    model=FAST_MODEL,
    description="Checks grocery items.",
    instruction="""
    You are a product analyst. 
    1. Search for specific safe brands/products.
    2. SCORING RUBRIC:
       - 95-100: "Certified Gluten Free" / Allergen Free.
       - 85-94: Ingredients list is safe, "No Gluten" claim.
       - <85: May contain traces.
    3. Output JSON.
    """
)

auditor_agent = LlmAgent(
    name="safety_auditor",
    model=SMART_MODEL,
    description="Evaluates the safety report.",
    instruction="Review recommendations. Assign Score (0-100), Headline, and 3-4 Bullet points. Output JSON."
)

In [4]:
def process_user_request(user_query, user_profile, location):
    print(f"🤖 Processing Query: '{user_query}' in '{location}'")
    print(f"👤 Profile: {user_profile}\n")
    
    results = {
        "intent": "",
        "recommendations": [], 
        "audit": {}
    }

    # Step 1: Route
    print("🔀 Step 1: Routing...")
    intent = query_agent_with_runner(router_agent, f"Query: {user_query}").strip()
    if "RESTAURANT" in intent.upper(): intent = "RESTAURANT"
    elif "GROCERY" in intent.upper(): intent = "GROCERY"
    results["intent"] = intent
    print(f"   -> Intent Detected: {intent}")

    # Step 2: Vet
    print("🕵️ Step 2: Vetting Candidates (Google Search)...")
    localized_prompt = f"""
    User Query: {user_query}
    Target City: {location}
    Dietary Profile: {user_profile}
    
    INSTRUCTIONS:
    1. Search strictly within {location}.
    2. Find up to 6 valid options. 
    """
    
    tools_to_use = [search_tool]
    
    if intent == "RESTAURANT":
        raw_json = query_agent_with_runner(restaurant_vetter, localized_prompt, tools=tools_to_use, json_mode=True, schema=rec_schema)
    else:
        raw_json = query_agent_with_runner(grocery_vetter, localized_prompt, tools=tools_to_use, json_mode=True, schema=rec_schema)
    
    try:
        results["recommendations"] = json.loads(raw_json)
        print(f"   -> Found {len(results['recommendations'])} candidates.")
    except:
        results["recommendations"] = []
        print("   -> ⚠️ Failed to parse vetting results.")

    # Step 3: Audit
    print("⚖️ Step 3: Auditing...")
    audit_json = query_agent_with_runner(
        auditor_agent, 
        f"Review list for profile '{user_profile}':\n{raw_json}",
        json_mode=True,
        schema=auditor_schema
    )

    try:
        results["audit"] = json.loads(audit_json)
        print(f"   -> Audit Complete. Score: {results['audit'].get('overall_score')}")
    except:
        results["audit"] = {"overall_score": 0, "headline": "Error", "summary_notes": []}
    
    return results

In [5]:
# --- DEMO RUN ---
profile = "Vegetarian. Eats Eggs (Ovo-Vegetarian). No meat broth. No gelatin."
location = "Edison, NJ"
query = "Good ramen place"

final_output = process_user_request(query, profile, location)

# Pretty Print Results
print("\n" + "="*40)
print(f"🍽️ FINAL REPORT: {final_output['intent']}")
print("="*40)
print(f"\n[AUDITOR VERDICT]: {final_output['audit'].get('headline')} ({final_output['audit'].get('overall_score')}%)")
for note in final_output['audit'].get('summary_notes', []):
    print(f" - {note}")

print(f"\n[RECOMMENDATIONS ({len(final_output['recommendations'])})]:")
for place in final_output['recommendations']:
    print(f"\n📍 {place['name']} ({place['safety_score']}% Safe)")
    print(f"   Address: {place['address']}")
    print(f"   Items: {', '.join(place['safe_items'][:3])}...")
    print(f"   Reasoning: {place['reasoning']}")

🤖 Processing Query: 'Good ramen place' in 'Edison, NJ'
👤 Profile: Vegetarian. Eats Eggs (Ovo-Vegetarian). No meat broth. No gelatin.

🔀 Step 1: Routing...
   -> Intent Detected: Okay, I can help you find a good ramen place! To give you the best recommendations, I need a little more information:

*   **Where are you located?** (City, neighborhood, or even a specific address)
*   **What kind of ramen are you in the mood for?** (e.g., Tonkotsu, Shoyu, Miso, Spicy, Vegetarian)
*   **What's your budget?** (e.g., cheap eats, mid-range, splurge)
*   **Are you looking for a specific atmosphere?** (e.g., casual, trendy, authentic, family-friendly)
*   **Do you have any dietary restrictions?** (e.g., vegetarian, vegan, gluten-free)

In the meantime, here are some general tips for finding good ramen:

*   **Check online reviews:** Look at Google Maps, Yelp, TripAdvisor, and other review sites. Pay attention to the number of reviews and the overall rating.
*   **Ask locals:** If you're in a new ar

## ☁️ Deployment Strategy (Bonus Implementation)
To move this agent from a notebook to a production web app, I have created the following files:

### 1. `Dockerfile`
This containerizes the application for Google Cloud Run.
```dockerfile
FROM python:3.11-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
EXPOSE 8080
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8080"]